# Salvar e carregar modelos

Author: Cláudia Magliano

Date: 29/07/2024

# Configurar
Instalar e importar o TensorFlow e as dependências

In [2]:
pip install pyyaml h5py  # Required to save models in HDF5 format

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.15.0


# Obter um conjunto de dados de exemplo
Para demonstrar como salvar e carregar pesos, você usará o conjunto de dados MNIST . Para acelerar essas execuções, use os primeiros 1.000 exemplos:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


# Definir um modelo
Começar construindo um modelo sequencial simples

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Salvar pontos de verificação durante o treinamento

Usar um modelo treinado sem precisar retreiná-lo ou retomar o treinamento de onde parou, caso o processo de treinamento tenha sido interrompido. O retorno de chamada tf.keras.callbacks.ModelCheckpoint permite que você salve continuamente o modelo durante e no final do treinamento.

Uso de retorno de chamada do ponto de verificação
Crie um retorno de chamada tf.keras.callbacks.ModelCheckpoint que economize pesos apenas durante o treinamento.

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 1.1795 - sparse_categorical_accuracy: 0.6649
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 4s 20ms/step - loss: 1.1303 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.7097 - val_sparse_categorical_accuracy: 0.7700
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4355 - sparse_categorical_accuracy: 0.8761
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.4279 - sparse_categorical_accuracy: 0.8770 - val_loss: 0.5201 - val_sparse_categorical_accuracy: 0.8360
Epoch 3/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2688 - sparse_categorical_accuracy: 0.9375
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.2746 - sparse_categorical_accuracy: 0.9360 - val_loss: 0.4735 - val_sparse_categorical_accuracy: 0.8520
Epoch 4/10
31/32 [========

Isso cria uma única coleção de arquivos de checkpoint do TensorFlow que são atualizados no final de cada época.

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

Desde que dois modelos compartilhem a mesma arquitetura, pode-se compartilhar pesos entre eles. Portanto, ao restaurar um modelo somente de pesos, crie um modelo com a mesma arquitetura do modelo original e, em seguida, defina seus pesos.

Agora reconstrua um modelo novo e não treinado e avalie-o no conjunto de teste. Um modelo não treinado funcionará em níveis de chance (~ 10% de precisão):

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.4214 - sparse_categorical_accuracy: 0.0880 - 150ms/epoch - 5ms/step
Untrained model, accuracy:  8.80%


Em seguida, carregue os pesos do ponto de verificação e reavalie:

In [8]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4076 - sparse_categorical_accuracy: 0.8720 - 77ms/epoch - 2ms/step
Restored model, accuracy: 87.20%


Opções de retorno de chamada de ponto de verificação
O retorno de chamada oferece várias opções para fornecer nomes exclusivos para pontos de verificação e ajustar a frequência do ponto de verificação.

Treine um novo modelo e salve pontos de verificação com nomes exclusivos uma vez a cada cinco épocas:

In [9]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


Agora, observe os checkpoints resultantes e escolha o mais recente:

In [10]:
os.listdir(checkpoint_dir)

['cp-0045.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0025.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

Para testar, redefina o modelo e carregue o último ponto de verificação:

In [12]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4820 - sparse_categorical_accuracy: 0.8800 - 147ms/epoch - 5ms/step
Restored model, accuracy: 88.00%


# O que são esses arquivos?
O código acima armazena os pesos em uma coleção de arquivos formatados de ponto de verificação que contêm apenas os pesos treinados em formato binário. Os pontos de verificação contêm:

Um ou mais fragmentos que contêm os pesos do seu modelo.
Um arquivo de índice que indica quais pesos são armazenados em qual estilhaço.
Se você estiver treinando um modelo em uma única máquina, terá um estilhaço com o sufixo: .data-00000-of-00001

# Salvar pesos manualmente
Salvando pesos manualmente com o método Model.save_weights . Por padrão, tf.keras — e save_weights em particular — usa o formato de checkpoint TensorFlow com uma extensão .ckpt (salvar em HDF5 com uma extensão .h5 é abordado no guia Salvar e serializar modelos ):

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# Salve todo o modelo
Chame model.save para salvar a arquitetura, os pesos e a configuração de treinamento de um modelo em um único arquivo/pasta. Isso permite exportar um modelo para que possa ser usado sem acesso ao código Python original*. Como o estado do otimizador é recuperado, você pode retomar o treinamento exatamente de onde parou.

Um modelo inteiro pode ser salvo em dois formatos de arquivo diferentes ( SavedModel e HDF5 ). O formato TensorFlow SavedModel é o formato de arquivo padrão no TF2.x. No entanto, os modelos podem ser salvos no formato HDF5 . Mais detalhes sobre como salvar modelos inteiros nos dois formatos de arquivo são descritos abaixo.

Salvar um modelo totalmente funcional é muito útil — você pode carregá-los no TensorFlow.js ( Saved Model , HDF5 ) e treiná-los e executá-los em navegadores da Web ou convertê-los para execução em dispositivos móveis usando o TensorFlow Lite ( Saved Model , HDF5 )

*Os objetos personalizados (por exemplo, modelos ou camadas subclassificados) requerem atenção especial ao salvar e carregar. Consulte a seção Salvando objetos personalizados abaixo

# Formato SavedModel
O formato SavedModel é outra maneira de serializar modelos. Os modelos salvos nesse formato podem ser restaurados usando tf.keras.models.load_model e são compatíveis com o TensorFlow Serving. O guia SavedModel detalha como servir/inspecionar o SavedModel. A seção abaixo ilustra as etapas para salvar e restaurar o modelo.

In [13]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 3ms/step - loss: 1.1026 - sparse_categorical_accuracy: 0.6840
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4175 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2668 - sparse_categorical_accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2046 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1597 - sparse_categorical_accuracy: 0.9630


O formato SavedModel é um diretório que contém um binário protobuf e um ponto de verificação do TensorFlow. Inspecione o diretório do modelo salvo:

In [18]:
# Contains an assets folder, saved_model.pb, and variables folder.
os.listdir('saved_model/my_model')

['variables',
 'saved_model.pb',
 'keras_metadata.pb',
 'assets',
 'fingerprint.pb']

Recarregue um novo modelo Keras do modelo salvo:

In [19]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


O modelo restaurado é compilado com os mesmos argumentos do modelo original. Tente executar avaliar e prever com o modelo carregado:

In [20]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4546 - sparse_categorical_accuracy: 0.8530 - 161ms/epoch - 5ms/step
Restored model, accuracy: 85.30%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


# Formato HDF5
Keras fornece um formato básico de salvamento usando o padrão HDF5 .

In [21]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 3ms/step - loss: 1.0988 - sparse_categorical_accuracy: 0.6970
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4207 - sparse_categorical_accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2867 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2076 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1476 - sparse_categorical_accuracy: 0.9680


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Agora, recrie o modelo desse arquivo:

In [22]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Verifique sua precisão:

In [23]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4311 - sparse_categorical_accuracy: 0.8610 - 155ms/epoch - 5ms/step
Restored model, accuracy: 86.10%


# Keras salva modelos inspecionando suas arquiteturas. Esta técnica salva tudo:

Os valores de peso
A arquitetura do modelo
A configuração de treinamento do modelo (o que você passa para o método .compile() )
O otimizador e seu estado, se houver (isso permite que você reinicie o treinamento de onde parou)
Keras não pode salvar os otimizadores v1.x (de tf.compat.v1.train ) porque eles não são compatíveis com checkpoints. Para otimizadores v1.x, você precisa recompilar o modelo após o carregamento — perdendo o estado do otimizador.



# Salvando objetos personalizados
Se estiver usando o formato SavedModel, pule esta seção. A principal diferença entre HDF5 e SavedModel é que HDF5 usa configurações de objeto para salvar a arquitetura do modelo, enquanto SavedModel salva o gráfico de execução. Assim, SavedModels são capazes de salvar objetos personalizados como modelos de subclasses e camadas personalizadas sem exigir o código original.

Para salvar objetos personalizados em HDF5, você deve fazer o seguinte:

Defina um método get_config em seu objeto e, opcionalmente, um método de from_config .
get_config(self) retorna um dicionário serializável JSON de parâmetros necessários para recriar o objeto.
from_config(cls, config) usa a configuração retornada de get_config para criar um novo objeto. Por padrão, esta função usará a configuração como kwargs de inicialização ( return cls(**config) ).
Passe o objeto para o argumento custom_objects ao carregar o modelo. O argumento deve ser um dicionário mapeando o nome da classe string para a classe Python. Por exemplo tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})

Consulte o tutorial Escrevendo camadas e modelos do zero para obter exemplos de objetos personalizados e get_config .

In [ ]:
# MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.